In [1]:
ADD = 0x01
MUL = 0x02
SUB = 0x03
DIV = 0x04
SDIV = 0x05
MOD = 0x06
SMOD = 0x07
ADDMOD = 0x08
MULMOD = 0x09
EXP = 0x0A
SIGNEXTEND = 0x0B
LT = 0x10
GT = 0x11
SLT = 0x12
SGT = 0x13
EQ = 0x14
ISZERO = 0x15
AND = 0x16
OR = 0x17
XOR = 0x18
NOT = 0x19
BYTE = 0x1A
SHL = 0x1B
SHR = 0x1C
SAR = 0x1D
PUSH0 = 0x5F
PUSH1 = 0x60
PUSH32 = 0x7F
POP = 0x50
MLOAD = 0x51
MSTORE = 0x52
MSTORE8 = 0x53
SLOAD = 0x54
SSTORE = 0x55
MSIZE = 0x59

class EVM:
    def __init__(self, code):
        self.code = code            # Initialize bytecode, bytes object
        self.pc = 0                 # Initialize the program counter to 0
        self.stack = []             # The stack is initially empty
        self.memory = bytearray()   # Initialize the memory to empty
        self.storage = {}           # Initialize the storage to an empty dictionary

    def next_instruction(self):
        op = self.code[self.pc]     # Get the current command
        self.pc += 1                # Increment
        return op

    def push(self, size):
        data = self.code[self.pc:self.pc + size]    # Get data from code according to size
        value = int.from_bytes(data, 'big')         # Convert bytes to int
        self.stack.append(value)                    # Push into the stack
        self.pc += size                             # pc increases the size unit

    def pop(self):
        if len(self.stack) == 0:
            raise Exception('Stack underflow')
        return self.stack.pop()                     # Pop the stack

    def add(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a + b) % (2**256)                    # The addition result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)
        
    def mul(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a * b) % (2**256)                    # The multiplication result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)

    def sub(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a - b) % (2**256)                    # The result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)

    def div(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        if a == 0:
            res = 0
        else:
            res =  (a // b) % (2**256)
        self.stack.append(res)

    def sdiv(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = a//b % (2**256) if a!=0 else 0
        self.stack.append(res)

    def mod(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = a % b if a != 0 else 0
        self.stack.append(res)

    def smod(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = a % b if a != 0 else 0
        self.stack.append(res)

    def addmod(self):
        if len(self.stack) < 3:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        n = self.stack.pop()
        res = (a + b) % n if n != 0 else 0
        self.stack.append(res)

    def mulmod(self):
        if len(self.stack) < 3:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        n = self.stack.pop()
        res = (a * b) % n if n != 0 else 0
        self.stack.append(res)

    def exp(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = pow(a, b) % (2**256)
        self.stack.append(res)
        
    def signextend(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        b = self.stack.pop()
        x = self.stack.pop()
        if b < 32:                              # If b>=32, no expansion is required
            sign_bit = 1 << (8 * b - 1)         # The mask value corresponding to the highest bit (sign bit) of the b byte will be used to detect whether the sign bit of x is 1
            x = x & ((1 << (8 * b)) - 1)        # Perform a mask operation on x, retain the value of the first b+1 bytes of x, and set all the remaining bytes to 0
            if x & sign_bit:                    # Check if the sign bit of x is 1
                x = x | ~((1 << (8 * b)) - 1)   # Set the rest of x to 1
        self.stack.append(x)
        
    def lt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b < a))           # Note the comparison order here

    def gt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b > a))           # Note the comparison order here

    def slt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b < a))           # The values ​​in the minimalist evm stack are already stored as signed integers, so the implementation is the same as lt

    def sgt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b > a))           # The values ​​in the minimalist evm stack are already stored as signed integers, so the implementation is the same as gt

    def eq(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(a == b))

    def iszero(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        self.stack.append(int(a == 0))

    def and_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a & b)

    def or_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a | b)

    def xor_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a ^ b)

    def not_op(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        self.stack.append(~a % (2**256))            # The result of the bitwise NOT operation needs to be modulo 2^256 to prevent overflow
    def byte_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        position = self.stack.pop()
        value = self.stack.pop()
        if position >= 32:
            res = 0
        else:
            res = (value // pow(256, 31 - position)) & 0xFF
        self.stack.append(res)

    def shl(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append((b << a) % (2**256))      # The result of the left shift operation needs to be modulo 2^256
    
    def shr(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(b >> a)                   # Right shift operation
        
    def sar(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(b >> a)                   # Right shift operation

    def mstore(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        offset = self.stack.pop()
        value = self.stack.pop()
        while len(self.memory) < offset + 32:
            self.memory.append(0)                   # Memory Expansion
        self.memory[offset:offset+32] = value.to_bytes(32, 'big')

    def mstore8(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        offset = self.stack.pop()
        value = self.stack.pop()
        while len(self.memory) < offset + 32:
            self.memory.append(0)                   # Memory Expansion
        self.memory[offset] = value & 0xFF          # Take the least significant byte

    def mload(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        offset = self.stack.pop()
        while len(self.memory) < offset + 32:
            self.memory.append(0)                   # Memory Expansion
        value = int.from_bytes(self.memory[offset:offset+32], 'big')
        self.stack.append(value)

    def sload(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        key = self.stack.pop()
        value = self.storage.get(key, 0)            # If the key does not exist, return 0
        self.stack.append(value)

    def sstore(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        key = self.stack.pop()
        value = self.stack.pop()
        self.storage[key] = value

    def msize(self):
        self.stack.append(len(self.memory))

    def run(self):
        while self.pc < len(self.code):
            op = self.next_instruction()

            if PUSH1 <= op <= PUSH32:               # If it is PUSH1-PUSH32
                size = op - PUSH1 + 1
                self.push(size)
            elif op == PUSH0:                       # If it is PUSH0
                self.stack.append(0)
            elif op == POP:                         # If it is POP
                self.pop()
            elif op == ADD:                         # Process ADD instruction
                self.add()
            elif op == MUL:                         # Processing MUL instruction
                self.mul()
            elif op == SUB:                         # Processing SUB instructions
                self.sub()
            elif op == DIV:                         # Processing DIV directives
                self.div()
            elif op == SDIV:
                self.sdiv()
            elif op == MOD:
                self.mod()
            elif op == SMOD:
                self.smod()
            elif op == ADDMOD:
                self.addmod()
            elif op == MULMOD:
                self.mulmod()
            elif op == EXP:
                self.exp()
            elif op == SIGNEXTEND:
                self.signextend()
            elif op == LT:
                self.lt()
            elif op == GT:
                self.gt()
            elif op == SLT:
                self.slt()
            elif op == SGT:
                self.sgt()
            elif op == EQ:
                self.eq()
            elif op == ISZERO:
                self.iszero()
            elif op == AND:         # Processing AND instructions
                self.and_op()
            elif op == OR:          # Processing OR instructions
                self.or_op()
            elif op == XOR:         # Processing XOR instructions
                self.xor_op()
            elif op == NOT:         # Processing NOT instructions
                self.not_op()
            elif op == BYTE:        # Processing BYTE instructions
                self.byte_op()
            elif op == SHL:         # Processing SHL instructions
                self.shl()
            elif op == SHR:         # Processing SHR instructions
                self.shr()
            elif op == SAR:         # Processing SAR instructions
                self.sar()
            elif op == MLOAD:       # Processing MLOAD instructions
                self.mload()
            elif op == MSTORE:      # Processing MSTORE instructions
                self.mstore()
            elif op == MSTORE8:     # Processing MSTORE8 instructions
                self.mstore8()
            elif op == SLOAD: 
                self.sload()
            elif op == SSTORE:      # Processing SSTORE instructions
                self.sstore()
            elif op == MSIZE:       # Processing MSIZE instructions
                self.msize()
            else:
                raise Exception('Invalid opcode')

In [3]:
# SSTORE
code = b"\x60\x02\x60\x00\x55"
evm = EVM(code)
evm.run()
print(evm.storage)  
# Output: {0: 2}

{0: 2}


In [4]:
# SLOAD
code = b"\x60\x02\x60\x00\x55\x60\x00\x54"
evm = EVM(code)
evm.run()
print(evm.storage)  
# output: {0: 2}
print(evm.stack)  
# output: [2]

{0: 2}
[2]
